In [1]:
using CausalELM
using CSV
using DataFrames
using Random

In [2]:
pension_df = CSV.read("pension.csv", DataFrame)
pension_df = pension_df[:, [10, 22, 13, 14, 15, 18, 20, 17, 24, 33]]
covariates, treatment, outcome = pension_df[:, 3:end], pension_df[:, 2], pension_df[:, 1]

(9915×8 DataFrame
  Row │ age    inc    fsize  marr   twoearn  db     pira   hown  
      │ Int64  Int64  Int64  Int64  Int64    Int64  Int64  Int64 
──────┼──────────────────────────────────────────────────────────
    1 │    31  28146      5      1        0      0      0      1
    2 │    52  32634      5      0        0      0      0      1
    3 │    50  52206      3      1        1      0      1      1
    4 │    28  45252      4      1        1      0      0      0
    5 │    42  33126      3      0        0      1      0      1
    6 │    49  76860      6      1        1      1      0      1
    7 │    40  57477      4      1        1      1      0      1
    8 │    58  14637      1      0        0      0      0      0
  ⋮   │   ⋮      ⋮      ⋮      ⋮       ⋮       ⋮      ⋮      ⋮
 9909 │    28  31926      2      1        1      0      0      0
 9910 │    49  64215      4      1        1      0      1      1
 9911 │    34  13500      1      0        0      1      0      0
 9912 

In [3]:
covariates.inc = (covariates.inc .- minimum(covariates.inc))/(maximum(covariates.inc) - minimum(covariates.inc))
covariates.age = (covariates.age .- minimum(covariates.age))/(maximum(covariates.age) - minimum(covariates.age))
covariates.fsize = (covariates.fsize .- minimum(covariates.fsize))/(maximum(covariates.fsize) - minimum(covariates.fsize))
covariates.marr = (covariates.marr .- minimum(covariates.marr))/(maximum(covariates.marr) - minimum(covariates.marr))
covariates

Row,age,inc,fsize,marr,twoearn,db,pira,hown
,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64
1,0.153846,0.125821,0.333333,1.0,0,0,0,1
2,0.692308,0.144156,0.333333,0.0,0,0,0,1
3,0.641026,0.224115,0.166667,1.0,1,0,1,1
4,0.0769231,0.195705,0.25,1.0,1,0,0,0
5,0.435897,0.146166,0.166667,0.0,0,1,0,1
6,0.615385,0.324836,0.416667,1.0,1,1,0,1
7,0.384615,0.245649,0.25,1.0,1,1,0,1
8,0.846154,0.0706319,0.0,0.0,0,0,0,0
9,0.102564,0.0376875,0.25,0.0,0,0,0,0


In [16]:
dml = DoubleMachineLearning(covariates, treatment, outcome, num_machines=50)

DoubleMachineLearning([0.46153846153846156 0.33966565349544076 … 1.0 1.0; 0.10256410256410256 0.08505735856456516 … 0.0 0.0; … ; 0.6923076923076923 0.042308069418570446 … 0.0 0.0; 0.10256410256410256 0.17147514462202176 … 1.0 1.0], [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0  …  0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], [14248.0, 2300.0, 0.0, 33748.0, -800.0, 25398.0, -1200.0, 120000.0, 15300.0, 100.0  …  60201.0, 51987.0, 9249.0, 6420.0, 3200.0, 99300.0, 19599.0, 8030.0, 4190.0, 8400.0], "ATE", false, "regression", CausalELM.swish, 9915, 50, 6, 24, NaN, 5)

In [5]:
r_learner = RLearner(covariates, treatment, outcome)

RLearner([0.3333333333333333 0.09809785273065987 … 1.0 0.0; 0.1282051282051282 0.08584174919109716 … 0.0 1.0; … ; 0.48717948717948717 0.6506030002941465 … 1.0 1.0; 0.5128205128205128 0.07530150014707324 … 0.0 1.0], [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0  …  0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0], [11000.0, 50.0, 157973.0, 100.0, -3700.0, 26000.0, 44.0, 32000.0, -6705.0, 10500.0  …  999.0, -18000.0, 46099.0, 920.0, -19950.0, 300.0, 11750.0, 182500.0, 47000.0, 499.0], "CATE", false, "regression", CausalELM.swish, 9915, 100, 6, 32, [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN], 5)

In [6]:
dre = DoublyRobustLearner(covariates, treatment, outcome)

DoublyRobustLearner([0.8974358974358975 0.2030100990293166 … 0.0 1.0; 0.5897435897435898 0.2634326894793607 … 0.0 1.0; … ; 0.0 0.19087655652514954 … 0.0 0.0; 0.3333333333333333 0.32516668300813806 … 1.0 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0], [41649.0, 9000.0, 0.0, 16350.0, 6000.0, 700.0, 13059.0, 5930.0, 23397.0, 1323.0  …  24500.0, 8050.0, -11000.0, 35499.0, -2854.0, 197590.0, -1400.0, 7700.0, 12000.0, 42050.0], "CATE", false, "regression", CausalELM.swish, 9915, 100, 6, 32, [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN], 2)

In [25]:
estimate_causal_effect!(dml)

8868.651114858334

In [8]:
estimate_causal_effect!(r_learner)

9915-element Vector{Float64}:
   7969.024541481493
   2551.07486621794
  48185.11603976369
   6562.417861484062
  12324.513387722585
  91413.60918565083
 103742.23330057286
  13234.161144429849
  16753.004994337723
   6429.458448880052
      ⋮
   2331.601849423459
  50477.892771963685
  19942.337555990453
  12658.185171498155
   -442.6517574940871
  72754.7346983037
  42410.30074258264
  64041.35045474993
   1374.0969545336325

In [9]:
estimate_causal_effect!(dre)

9915-element Vector{Float64}:
 13549.633020274861
 20881.59369086071
  1879.2141524564345
  4752.192233979611
  9972.464441326127
  5368.174090907391
  8080.56176700674
 11685.092957657413
 -1689.8961993687453
  4964.903827056494
     ⋮
 12745.035572594325
 13779.898140138454
 15285.34382394138
  7686.997478984806
 10874.155814573602
  9104.438679085306
  5974.4691837941145
   -39.615643944068324
 -9482.093434774426

In [10]:
summarize(dml)

Dict{Any, Any} with 11 entries:
  "Activation Function"    => swish
  "Quantity of Interest"   => "ATE"
  "Sample Size"            => 9915
  "Number of Machines"     => 100
  "Causal Effect"          => 8701.76
  "Number of Neurons"      => 24
  "Task"                   => "regression"
  "Time Series/Panel Data" => false
  "Standard Error"         => NaN
  "p-value"                => NaN
  "Number of Features"     => 6

In [11]:
summarize(r_learner)

Dict{Any, Any} with 11 entries:
  "Activation Function"    => swish
  "Quantity of Interest"   => "CATE"
  "Sample Size"            => 9915
  "Number of Machines"     => 100
  "Causal Effect"          => [7969.02, 2551.07, 48185.1, 6562.42, 12324.5, 914…
  "Number of Neurons"      => 32
  "Task"                   => "regression"
  "Time Series/Panel Data" => false
  "Standard Error"         => NaN
  "p-value"                => NaN
  "Number of Features"     => 6

In [12]:
summarise(dre)

Dict{Any, Any} with 11 entries:
  "Activation Function"    => swish
  "Quantity of Interest"   => "CATE"
  "Sample Size"            => 9915
  "Number of Machines"     => 100
  "Causal Effect"          => [13549.6, 20881.6, 1879.21, 4752.19, 9972.46, 536…
  "Number of Neurons"      => 32
  "Task"                   => "regression"
  "Time Series/Panel Data" => false
  "Standard Error"         => NaN
  "p-value"                => NaN
  "Number of Features"     => 6

In [13]:
validate(dml)

(Dict("0.1 Standard Deviations from Observed Outcomes" => -1974.20426849962, "0.075 Standard Deviations from Observed Outcomes" => -549.8183509860896, "0.025 Standard Deviations from Observed Outcomes" => -4377.799707458391, "0.05 Standard Deviations from Observed Outcomes" => -2591.878868163885), 2.7460736072464016, Matrix{Float64}(undef, 0, 9))

In [14]:
validate(r_learner)

(Dict("0.1 Standard Deviations from Observed Outcomes" => 248206.3227597734, "0.075 Standard Deviations from Observed Outcomes" => 404160.7518203919, "0.025 Standard Deviations from Observed Outcomes" => 322479.1870944485, "0.05 Standard Deviations from Observed Outcomes" => 155068.1882045497), 2.5694922346983624, Matrix{Float64}(undef, 0, 9))

In [15]:
validate(dre)

(Dict("0.1 Standard Deviations from Observed Outcomes" => 430.7554873780964, "0.075 Standard Deviations from Observed Outcomes" => -4156.750735846773, "0.025 Standard Deviations from Observed Outcomes" => -5301.764975883297, "0.05 Standard Deviations from Observed Outcomes" => -6012.136217190272), 2.5976021674608534, Matrix{Float64}(undef, 0, 9))